In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime as dt
import random

In [2]:
pd.set_option('display.max_colwidth', None, "display.max_rows", None, "display.max_columns", None)
pd.options.mode.chained_assignment = None

In [3]:
config = "1tls_3x3"

In [4]:
df_dqn_con = pd.read_csv(f'./{config}/con/DuelingDoubleDQNAgent_lr0.0001.csv', delimiter=',').assign(con=True)
df_maxpressure_con = pd.read_csv(f'./{config}/con/MaxPressureBaseline.csv', delimiter=',').assign(con=True)
df_sotl_con = pd.read_csv(f'./{config}/con/SotlBaseline.csv', delimiter=',').assign(con=True)
df_uniform_con = pd.read_csv(f'./{config}/con/UniformBaseline.csv', delimiter=',').assign(con=True)
"""
df_dqn_def = pd.read_csv(f'./{config}/def/DuelingDoubleDQNAgent_lr0.0001.csv', delimiter=',').assign(con=False)
df_maxpressure_def = pd.read_csv(f'./{config}/def/MaxPressureBaseline.csv', delimiter=',').assign(con=False)
df_sotl_def = pd.read_csv(f'./{config}/def/SotlBaseline.csv', delimiter=',').assign(con=False)
df_uniform_def = pd.read_csv(f'./{config}/def/UniformBaseline.csv', delimiter=',').assign(con=False)
"""

"\ndf_dqn_def = pd.read_csv(f'./{config}/def/DuelingDoubleDQNAgent_lr0.0001.csv', delimiter=',').assign(con=False)\ndf_maxpressure_def = pd.read_csv(f'./{config}/def/MaxPressureBaseline.csv', delimiter=',').assign(con=False)\ndf_sotl_def = pd.read_csv(f'./{config}/def/SotlBaseline.csv', delimiter=',').assign(con=False)\ndf_uniform_def = pd.read_csv(f'./{config}/def/UniformBaseline.csv', delimiter=',').assign(con=False)\n"

In [5]:
dfs = [df_dqn_con, df_maxpressure_con, df_sotl_con, df_uniform_con] # , df_dqn_def, df_maxpressure_def, df_sotl_def, df_uniform_def]

In [6]:
for df in dfs:
    display(df.loc[df["done"] == True].head(2))

,l,r,id,ep,con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_queue_length,TimeLimit.truncated,done,con
299,300,84.738664,rlcontroller,1,0.3,"[1000.0, 200.0, 500.0, 200.0]",37,30.508629,3763.0,22,0.824558,101.702703,1.833333,False,True,True
617,318,114.218432,rlcontroller,2,0.1,"[200.0, 200.0, 600.0, 300.0]",81,73.429016,7054.0,52,0.906531,87.086420,4.333333,False,True,True


,l,r,id,ep,con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_queue_length,TimeLimit.truncated,done,con
263,264,0.0,maxpressurebaseline,1,0.3,"[1000.0, 200.0, 500.0, 200.0]",27,19.034108,464.0,12,0.704967,17.185185,1.000000,False,True,True
545,282,0.0,maxpressurebaseline,2,0.1,"[200.0, 200.0, 600.0, 300.0]",41,34.096772,4592.0,28,0.831629,112.000000,2.333333,False,True,True


,l,r,id,ep,con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_queue_length,TimeLimit.truncated,done,con
292,293,0.0,sotlbaseline,1,0.3,"[1000.0, 200.0, 500.0, 200.0]",189,183.000498,4860.0,144,0.968257,25.714286,12.00,False,True,True
611,319,0.0,sotlbaseline,2,0.1,"[200.0, 200.0, 600.0, 300.0]",117,112.412626,1165.0,81,0.960792,9.957265,6.75,False,True,True


,l,r,id,ep,con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_queue_length,TimeLimit.truncated,done,con
239,240,0.0,uniformbaseline,1,0.3,"[1000.0, 200.0, 500.0, 200.0]",110,102.707175,2078.0,87,0.933702,18.890909,7.250000,False,True,True
479,240,0.0,uniformbaseline,2,0.1,"[200.0, 200.0, 600.0, 300.0]",94,89.115024,1436.0,71,0.948032,15.276596,5.916667,False,True,True


In [7]:
def sum_ep(df):
    return pd.merge(
        df[['id', 'ep', 'veh_n', 'sum_delay', 'sum_waiting_time', 'sum_queue_length']].groupby(['ep']).sum(),
        df[['id', 'ep', 'con_p_rate', 'veh_n_p_hour', 'con']].groupby(['ep']).apply(lambda df: df.head(1)),
        how='left', left_index=True, right_index=True
    ).reset_index(drop=True)

def sum_df_ep(dfs):
    return pd.concat([sum_ep(df) for df in dfs]).sort_values(['ep', 'id']).reset_index(drop=True)

In [8]:
def avg_ep(df):
    return pd.merge(
        df[['id', 'ep', 'veh_n', 'sum_delay', 'sum_waiting_time', 'sum_queue_length']].groupby(['ep']).mean(),
        df[['id', 'ep', 'con_p_rate', 'veh_n_p_hour', 'con']].groupby(['ep']).apply(lambda df: df.head(1)),
        how='left', left_index=True, right_index=True
    ).reset_index(drop=True)

def avg_df_ep(dfs):
    return pd.concat([avg_ep(df) for df in dfs]).sort_values(['ep', 'id']).reset_index(drop=True)

In [9]:
sum_df = sum_df_ep(dfs)
avg_df = avg_df_ep(dfs)

In [10]:
display(sum_df.head(12))
display(avg_df.head(12))

,veh_n,sum_delay,sum_waiting_time,sum_queue_length,id,ep,con_p_rate,veh_n_p_hour,con
0,7452,5476.671238,162262.0,3218,maxpressurebaseline,1,0.3,"[1000.0, 200.0, 500.0, 200.0]",True
1,12231,9974.559985,639475.0,6944,rlcontroller,1,0.3,"[1000.0, 200.0, 500.0, 200.0]",True
2,39477,37596.127634,1260712.0,28667,sotlbaseline,1,0.3,"[1000.0, 200.0, 500.0, 200.0]",True
3,21163,19630.189134,287377.0,14062,uniformbaseline,1,0.3,"[1000.0, 200.0, 500.0, 200.0]",True
4,10442,8703.617460,721067.0,6410,maxpressurebaseline,2,0.1,"[200.0, 200.0, 600.0, 300.0]",True
5,21940,19986.314203,7174509.0,17016,rlcontroller,2,0.1,"[200.0, 200.0, 600.0, 300.0]",True
6,31712,30127.628536,331061.0,21174,sotlbaseline,2,0.1,"[200.0, 200.0, 600.0, 300.0]",True
7,18717,17396.580401,260579.0,12590,uniformbaseline,2,0.1,"[200.0, 200.0, 600.0, 300.0]",True
8,21411,18669.528696,521219.0,12757,maxpressurebaseline,3,0.4,"[1000.0, 900.0, 300.0, 600.0]",True
9,15538,12359.924345,369449.0,7921,rlcontroller,3,0.4,"[1000.0, 900.0, 300.0, 600.0]",True


,veh_n,sum_delay,sum_waiting_time,sum_queue_length,id,ep,con_p_rate,veh_n_p_hour,con
0,28.227273,20.744967,614.628788,12.189394,maxpressurebaseline,1,0.3,"[1000.0, 200.0, 500.0, 200.0]",True
1,40.770000,33.248533,2131.583333,23.146667,rlcontroller,1,0.3,"[1000.0, 200.0, 500.0, 200.0]",True
2,134.733788,128.314429,4302.771331,97.839590,sotlbaseline,1,0.3,"[1000.0, 200.0, 500.0, 200.0]",True
3,88.179167,81.792455,1197.404167,58.591667,uniformbaseline,1,0.3,"[1000.0, 200.0, 500.0, 200.0]",True
4,37.028369,30.863892,2556.975177,22.730496,maxpressurebaseline,2,0.1,"[200.0, 200.0, 600.0, 300.0]",True
5,68.993711,62.850045,22561.349057,53.509434,rlcontroller,2,0.1,"[200.0, 200.0, 600.0, 300.0]",True
6,99.410658,94.443977,1037.808777,66.376176,sotlbaseline,2,0.1,"[200.0, 200.0, 600.0, 300.0]",True
7,77.987500,72.485752,1085.745833,52.458333,uniformbaseline,2,0.1,"[200.0, 200.0, 600.0, 300.0]",True
8,84.295276,73.502081,2052.043307,50.224409,maxpressurebaseline,3,0.4,"[1000.0, 900.0, 300.0, 600.0]",True
9,53.030717,42.184042,1260.918089,27.034130,rlcontroller,3,0.4,"[1000.0, 900.0, 300.0, 600.0]",True
